### Plan de ce notebook

1. Use pre-trained GloVe words for embeddings
2. Use pre-trained word2vec words for embeddings
3. What about Freebase ?

pour standardiser le nom des variables: GloVe _stanford et word2vec _google ?

# Nettoyage et conversion numérique des données

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GlobalMaxPooling1D, Bidirectional, Conv1D, concatenate
from keras.models import Model

from tools import *

Using TensorFlow backend.


In [2]:
# load raw string data
data_train, y_train_all, data_test, id_test = load_data()

## Nettoyage des données (optionnel)

In [3]:
params = {'lower': True, 
          'lemma': False, 
          'stop_words': False}

comment = data_train[2]
print(comment)
print('-------')
print(clean_comment(comment, **params))

Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.
-------
hey man i m really not trying to edit war it s just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info 


In [4]:
clean_data_train = transform_dataset(data_train, func=clean_comment, kwargs=params)
clean_data_test = transform_dataset(data_test, func=clean_comment, kwargs=params)

Transformation: 100%       
Transformation: 100%       


## Tokenization et découpage des données textuelles 

Conforme au github https://github.com/msahamed/yelp_comments_classification_nlp/blob/master/word_embeddings.ipynb

pour pouvoir encode avec Glove après (ne pas encode directement != Nicolas sur models_testing.ipynb)

In [5]:
# Convert strings to int indexes, 
# considering only the VOCAB_SIZE most commons words, 
# and pad the sentences to SENTENCE_LENGTH words
VOCAB_SIZE = 30000
SENTENCE_LENGTH = 200  # 200 if stop_words deleted, 120 otherwise

In [6]:
tokenizer = TokenVectorizer(max_len=SENTENCE_LENGTH, max_features=VOCAB_SIZE)

# X_train_all, X_test = encode(data_train, data_test, vectorizer=tokens_vectorizer)
X_train_all, X_test = encode(clean_data_train, clean_data_test, vectorizer=tokenizer)

ENCODING: Fitting vectorizer to data
ENCODING: transforming data to numerical


In [7]:
SPLIT_VALID_RATIO = 0.10
SPLIT_RANDOM_SEED = 0  # TODO : check split because of unbalanced classes

X_train, X_valid, y_train, y_valid = train_test_split(X_train_all, y_train_all, 
                                                      test_size=SPLIT_VALID_RATIO,
                                                      random_state=SPLIT_RANDOM_SEED)

# 1. Use pre-trained GloVe words for embeddings

https://medium.com/@sabber/classifying-yelp-review-comments-using-cnn-lstm-and-pre-trained-glove-word-embeddings-part-3-53fcea9a17fa

https://github.com/msahamed/yelp_comments_classification_nlp

https://github.com/msahamed/yelp_comments_classification_nlp/blob/master/word_embeddings.ipynb

## embedding_matrix avec les embeddings de Glove

several possibilities of pre-training/embeddings vector sizes for GloVe, see:

https://nlp.stanford.edu/projects/glove/

dans Mikolov c'est du 300 pour la taille des embeddings https://code.google.com/archive/p/word2vec/

In [16]:
# Load GloVe pre-trained embeddings
EMBEDDING_DIM = 200  # several embeddings sizes possible with Glove : 50, 100, 200, 300 
GLOVE_EMBEDDINGS = 'embeddings/glove.6B.{}d.txt'.format(EMBEDDING_DIM)

embeddings_index = dict()
with open(GLOVE_EMBEDDINGS) as file:
    for line in file:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        #print(coefs.shape) # le code est bien compatible avec le format renvoyé par word2vec
        embeddings_index[word] = coefs
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [17]:
# Create a weight matrix for words in training docs

# get mean and std values of pre-trained embeddings
all_embs = np.stack(embeddings_index.values())
emb_mean, emb_std = np.mean(all_embs, axis=0), np.std(all_embs, axis=0)

# init matrix to embeddings distribution
embedding_matrix = np.random.normal(emb_mean, emb_std, (VOCAB_SIZE, EMBEDDING_DIM))
# loop on words in our documents
for word, index in tokenizer.word_index.items():
    # if word isn't enough used in documents, ignore it
    if index >= VOCAB_SIZE: 
        continue
    # otherwise, fill embedding matrix with pre-trained vector corresponding to this word
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[index, :] = embedding_vector

## Définition du réseau de Yoon Kim pour GloVe

In [24]:
N_FILTERS = 100
MODEL_NAME = "embed_conv_fc_GLOVE200t"

# input
inp = Input(shape=(SENTENCE_LENGTH, ))
# embedding
emb = Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=SENTENCE_LENGTH, 
                weights=[embedding_matrix], trainable=True)(inp)

# Specify each convolution layer and their kernel siz i.e. n-grams 
conv_3 = Conv1D(filters=N_FILTERS, kernel_size=3, activation='relu')(emb)
pool_3 = GlobalMaxPooling1D()(conv_3)

conv_4 = Conv1D(filters=N_FILTERS, kernel_size=4, activation='relu')(emb)
pool_4 = GlobalMaxPooling1D()(conv_4)

conv_5 = Conv1D(filters=N_FILTERS, kernel_size=5, activation='relu')(emb)
pool_5 = GlobalMaxPooling1D()(conv_5)

# Gather all convolution layers
x = concatenate([pool_3, pool_4, pool_5], axis=1)
x = Dropout(0.1)(x)
x = Dense(50, activation='relu')(x)
x = Dropout(0.1)(x)
outp = Dense(6, activation='sigmoid')(x)

# # load pre-trained model from disk
# model = load_nnet(MODEL_NAME)

model = Model(inputs=inp, outputs=outp)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [25]:
# train
BATCH_SIZE = 32
N_EPOCHS = 2

RocAuc = RocAucEvaluation(validation_data=(X_valid, y_valid))

hist = model.fit(X_train, y_train, 
                 batch_size=BATCH_SIZE, 
                 epochs=N_EPOCHS, 
                 validation_data=(X_valid, y_valid),
                 callbacks=[RocAuc])

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 1203s 8ms/step - loss: 0.0544 - acc: 0.9800 - val_loss: 0.0443 - val_acc: 0.9831
epoch: 1 - val_roc_auc: 0.9846
Epoch 2/2
143613/143613 [==============================] - 1242s 9ms/step - loss: 0.0407 - acc: 0.9838 - val_loss: 0.0445 - val_acc: 0.9831
epoch: 2 - val_roc_auc: 0.9858


In [26]:
# save trained nnet to disk for later use
save_nnet(model, MODEL_NAME)

In [27]:
# final model evaluation
y_train_pred = model.predict(X_train, batch_size=512)
train_score = evaluate(y_train, y_train_pred)
print("ROC-AUC score on train set : {:.4f}".format(train_score)) 

y_valid_pred = model.predict(X_valid, batch_size=512)
valid_score = evaluate(y_valid, y_valid_pred)
print("ROC-AUC score on validation set : {:.4f}".format(valid_score))

ROC-AUC score on train set : 0.9944
ROC-AUC score on validation set : 0.9858


In [28]:
# predict
y_test_pred = model.predict(X_test, batch_size=512, verbose=2)

In [29]:
# write submission file
submission(y_test_pred, id_test, name=MODEL_NAME)

# 2. Use pre-trained word2vec words for embeddings

le github précédent utilise word2vec pour entraîner sur le corpus du problème même, et pas comme source d'embeddings pré entraînés... https://github.com/msahamed/yelp_comments_classification_nlp/blob/master/word_embeddings.ipynb

"In this subsection, I use word2vec to create word embeddings from the review comments. Word2vec is one algorithm for learning a word embedding from a text corpus." --->>> à la base word2vec c'est le réseau d'extraction, on veut récupérer un résultat d'entraînement de référence de cette architecture !

Il faut utiliser word2vec entraîné sur Google News, embeddings de taille 300, par Mikolov https://code.google.com/archive/p/word2vec/

Post de blog chargeant word2vec pré-entraîné par Mikolov:

https://machinelearningmastery.com/develop-word-embeddings-python-gensim/

Lien de téléchargement des embeddings pré entraînés:

https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM&export=download

In [3]:
from gensim.models import KeyedVectors

filename = 'GoogleNews-vectors-negative300.bin/data'
google_model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [8]:
google_model_words = list(google_model.wv.vocab)
print("Number of words in pre trained word2vec:",len(google_model_words))

Number of words in pre trained word2vec: 3000000


/home/mbauw/Documents/Académique/M2_AIC/OPT1_DataCamp/mars_craters/mars_env/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [12]:
print(type(google_model['fuck']))

<class 'numpy.ndarray'>


In [13]:
google_model['fuck'].shape

(300,)

## embedding_matrix avec les embeddings de google news word2vec

on s'inspire de la construction de la matrice pour GloVe, 
ici pas besoin de construire le dictionnaire à partir du modèle

notons que le tokenizer est déjà initialisé et reste le même

In [25]:
# Constantes inchangées:
# VOCAB_SIZE = 30000
# SENTENCE_LENGTH = 200  # 200 if stop_words deleted, 120 otherwise

# ONLY ONE embedding size possible with google news trained word2vec
EMBEDDING_DIM_GOOGLE = 300

In [26]:
# create a weight matrix for words in training docs
embedding_matrix_google = np.zeros((VOCAB_SIZE, EMBEDDING_DIM_GOOGLE))
for word, index in tokenizer.word_index.items():
    if index > VOCAB_SIZE - 1: # détermine à quel point on s'intéresse aux mots moins importants d'après Glove
        break
    else:
        embedding_vector_google = google_model[word] # on va chercher le mot dans word2vec embeddings
        if embedding_vector is not None:
            embedding_matrix_google[index] = embedding_vector_google

In [27]:
embedding_matrix_google.shape

(30000, 300)

## Définition du réseau de Yoon Kim pour word2vec

In [29]:
N_FILTERS = 100
MODEL_NAME_GOOGLE = "embed_conv_fc_GOOGLE"

# input
inp = Input(shape=(SENTENCE_LENGTH, ))
# embedding
emb = Embedding(VOCAB_SIZE, EMBEDDING_DIM_GOOGLE,input_length=SENTENCE_LENGTH,weights=[embedding_matrix_google], trainable=False)(inp)

# Specify each convolution layer and their kernel siz i.e. n-grams 
conv_3 = Conv1D(filters=N_FILTERS, kernel_size=3, activation='relu')(emb)
pool_3 = GlobalMaxPooling1D()(conv_3)

conv_4 = Conv1D(filters=N_FILTERS, kernel_size=4, activation='relu')(emb)
pool_4 = GlobalMaxPooling1D()(conv_4)

conv_5 = Conv1D(filters=N_FILTERS, kernel_size=5, activation='relu')(emb)
pool_5 = GlobalMaxPooling1D()(conv_5)

# Gather all convolution layers
x = concatenate([pool_3, pool_4, pool_5], axis=1)
x = Dropout(0.1)(x)
x = Dense(50, activation='relu')(x)
x = Dropout(0.1)(x)
outp = Dense(6, activation='sigmoid')(x)

# # load pre-trained model from disk
# model = load_nnet(MODEL_NAME)

model_google = Model(inputs=inp, outputs=outp)
model_google.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [30]:
# train
BATCH_SIZE = 32
N_EPOCHS = 2

RocAuc = RocAucEvaluation(validation_data=(X_valid, y_valid))

hist = model_google.fit(X_train, y_train, 
                 batch_size=BATCH_SIZE, 
                 epochs=N_EPOCHS, 
                 validation_data=(X_valid, y_valid),
                 callbacks=[RocAuc])

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 1552s 11ms/step - loss: 0.2890 - acc: 0.9634 - val_loss: 0.1603 - val_acc: 0.9629
epoch: 1 - val_roc_auc: 0.5000
Epoch 2/2
143613/143613 [==============================] - 1083s 8ms/step - loss: 0.1464 - acc: 0.9634 - val_loss: 0.1431 - val_acc: 0.9629
epoch: 2 - val_roc_auc: 0.5000


In [34]:
# save trained nnet to disk for later use
save_nnet(model_google, MODEL_NAME_GOOGLE)

In [35]:
# final model evaluation
y_train_pred = model_google.predict(X_train, batch_size=512)
train_score = evaluate(y_train, y_train_pred)
print("ROC-AUC score on train set : {:.4f}".format(train_score)) 

y_valid_pred = model_google.predict(X_valid, batch_size=512)
valid_score = evaluate(y_valid, y_valid_pred)
print("ROC-AUC score on validation set : {:.4f}".format(valid_score))

ROC-AUC score on train set : 0.5000
ROC-AUC score on validation set : 0.5000


In [36]:
# predict
y_test_pred = model_google.predict(X_test, batch_size=512, verbose=2)

In [37]:
# write submission file
submission(y_test_pred, id_test, name=MODEL_NAME_GOOGLE)

# 3. What about Freebase ?

https://code.google.com/archive/p/word2vec/